## Installing Pytorch

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

## Building Fairseq

In [1]:
!git clone https://github.com/pytorch/fairseq.git
import os
os.chdir("fairseq/")
!pip install -r requirements.txt
%run -i 'setup.py' build develop

Cloning into 'fairseq'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 3326 (delta 39), reused 35 (delta 20), pack-reused 3255
Receiving objects: 100% (3326/3326), 3.29 MiB | 13.96 MiB/s, done.
Resolving deltas: 100% (2421/2421), done.
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.6
creating build/lib.linux-x86_64-3.6/tests
copying tests/test_sequence_generator.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_reproducibility.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_sequence_scorer.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_train.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_iterators.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_noising.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_binaries.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_dictionary.py -> build/lib.l

## Model for Ts


### Data

In [2]:
!git clone https://github.com/LauraMartinus/ukuxhumana.git

Cloning into 'ukuxhumana'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 1386 (delta 58), reused 50 (delta 19), pack-reused 1266
Receiving objects: 100% (1386/1386), 404.63 MiB | 22.34 MiB/s, done.
Resolving deltas: 100% (665/665), done.
Checking out files: 100% (308/308), done.


### Subword

In [3]:
!git clone https://github.com/rsennrich/subword-nmt

Cloning into 'subword-nmt'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 509 (delta 14), reused 10 (delta 2), pack-reused 481
Receiving objects: 100% (509/509), 216.68 KiB | 14.45 MiB/s, done.
Resolving deltas: 100% (301/301), done.


In [0]:
#os.chdir('../')
!ls

## Study


**4k**

In [0]:
os.chdir('../')
!cat fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.en fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.tn > combine.txt
!fairseq/subword-nmt/learn_bpe.py -s 4000 <combine.txt> entn.codes

!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.tn > train.tn
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.dev.tn > valid.tn
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.test.tn > test.tn

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang tn --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/entn

!mkdir -p checkpoint/fconv
!python train.py data-bin/entn \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checkpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/entn --path checkpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang tn

**8k**

In [0]:
#os.chdir('../')
!ls

In [0]:
os.chdir('../')
!cat fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.en fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.tn > combine.txt
!fairseq/subword-nmt/learn_bpe.py -s 8000 <combine.txt> entn.codes

!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.tn > train.tn
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.dev.tn > valid.tn
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.test.tn > test.tn

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang tn --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/entn

!mkdir -p checkpoint/fconv
!python train.py data-bin/entn \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checkpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/entn --path checkpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang tn

**12k**

In [0]:
os.chdir('../')
!cat fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.en fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.tn > combine.txt
!fairseq/subword-nmt/learn_bpe.py -s 12000 <combine.txt> entn.codes

!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.tn > train.tn
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.dev.tn > valid.tn
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.test.tn > test.tn

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang tn --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/entn

!mkdir -p checkpoint/fconv
!python train.py data-bin/entn \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checkpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/entn --path checkpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang tn

**16k**

In [0]:
#os.chdir('content/')
!ls

In [0]:
os.chdir('../')
!cat fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.en fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.tn > combine.txt
!fairseq/subword-nmt/learn_bpe.py -s 16000 <combine.txt> entn.codes

!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.tn > train.tn
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.dev.tn > valid.tn
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.test.tn > test.tn

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang tn --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/entn

!mkdir -p checkpoint/fconv
!python train.py data-bin/entn \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checkpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/entn --path checkpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang tn

**20k**

In [0]:
!ls

In [4]:
os.chdir('../')
!cat fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.en fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.tn > combine.txt
!fairseq/subword-nmt/learn_bpe.py -s 20.000 <combine.txt> entn.codes

!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.tn > train.tn
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.dev.tn > valid.tn
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.test.tn > test.tn

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang tn --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/entn

!mkdir -p checkpoint/fconv
!python train.py data-bin/entn \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checkpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/entn --path checkpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang tn

fairseq/subword-nmt/learn_bpe.py:267: DeprecationWarning: this script's location has moved to /content/fairseq/subword-nmt/subword_nmt. This symbolic link will be removed in a future version. Please point to the new location, or install the package and use the command 'subword-nmt'
  DeprecationWarning
fairseq/subword-nmt/apply_bpe.py:334: DeprecationWarning: this script's location has moved to /content/fairseq/subword-nmt/subword_nmt. This symbolic link will be removed in a future version. Please point to the new location, or install the package and use the command 'subword-nmt'
  DeprecationWarning
fairseq/subword-nmt/apply_bpe.py:351: ResourceWarning: unclosed file <_io.TextIOWrapper name='entn.codes' mode='r' encoding='UTF-8'>
  args.codes = codecs.open(args.codes.name, encoding='utf-8')
fairseq/subword-nmt/apply_bpe.py:334: DeprecationWarning: this script's location has moved to /content/fairseq/subword-nmt/subword_nmt. This symbolic link will be removed in a future version. Pleas

**24k**

In [0]:
!ls

In [0]:
os.chdir('../')
!cat fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.en fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.tn > combine.txt
!fairseq/subword-nmt/learn_bpe.py -s 24000 <combine.txt> entn.codes

!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.tn > train.tn
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.dev.tn > valid.tn
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.test.tn > test.tn

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang tn --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/entn

!mkdir -p checkpoint/fconv
!python train.py data-bin/entn \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checkpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/entn --path checkpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang tn

**28k**

In [0]:
os.chdir('../')
!cat fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.en fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.tn > combine.txt
!fairseq/subword-nmt/learn_bpe.py -s 28000 <combine.txt> entn.codes

!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.tn > train.tn
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.dev.tn > valid.tn
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.test.tn > test.tn

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang tn --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/entn

!mkdir -p checkpoint/fconv
!python train.py data-bin/entn \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checkpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/entn --path checkpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang tn

**32k**

In [0]:
!ls

In [0]:
os.chdir('../')
!cat fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.en fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.tn > combine.txt
!fairseq/subword-nmt/learn_bpe.py -s 32000 <combine.txt> entn.codes

!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.tn > train.tn
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.dev.tn > valid.tn
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.test.tn > test.tn

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang tn --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/entn

!mkdir -p checkpoint/fconv
!python train.py data-bin/entn \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checkpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/entn --path checkpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang tn

**36k**

In [0]:
#os.chdir('fairseq/')
!ls

In [0]:
os.chdir('../')
!cat fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.en fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.tn > combine.txt
!fairseq/subword-nmt/learn_bpe.py -s 36000 <combine.txt> entn.codes

!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.tn > train.tn
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.dev.tn > valid.tn
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.test.tn > test.tn

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang tn --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/entn

!mkdir -p checkpoint/fconv
!python train.py data-bin/entn \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checkpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/entn --path checkpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang tn

**40k**

In [0]:
!ls

In [0]:
os.chdir('../')
!cat fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.en fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.tn > combine.txt
!fairseq/subword-nmt/learn_bpe.py -s 40000 <combine.txt> entn.codes

!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.train.tn > train.tn
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.dev.tn > valid.tn
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c entn.codes < fairseq/ukuxhumana/clean/en_tn/entn_parallel.test.tn > test.tn

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang tn --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/entn

!mkdir -p checkpoint/fconv
!python train.py data-bin/entn \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checkpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/entn --path checkpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang tn

## Results

*   40k: 
*   36k: 
*   32k: 
*   28k: 
*   24k: 
*   20k: 22.09
*   16k: 26.03
*   12k: 25.62
*     8k: 21.28
*     4k: 24.34

